In [29]:
from pyspark.sql import SparkSession
import os
from pyspark.sql import functions as F
# import configparser

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# song_s3_bucket = 

### Build Spark Session

In [2]:
spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
s3_bucket = 's3://test1dsa/'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Song data

In [3]:
# Just load one file
song_df = spark.read.json("s3://udacity-dend/song_data/A/B/*")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# 588 song in our dataframe
song_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

588

### Save songs table to S3

In [7]:
songs = song_df.select(['song_id', 'title', 'artist_id', 'year', 'duration']).distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
songs.write.mode("overwrite").partitionBy('year', 'artist_id').parquet(path='s3://test1dsa/songs/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Save Artists table to S3

In [21]:
# create view
artists_cols = ['artist_id', 'artist_name', 'artist_location','artist_latitude', 'artist_longitude']
artists_table = song_df.select(artists_cols).distinct()

# write
artists_table.write.mode("overwrite").parquet(path=s3_bucket + 'artists')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Log data

In [22]:
# Just load log event at November-2018
log_data_path = "s3://udacity-dend/log_data/2018/11/*"
log_df = spark.read.json(log_data_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
log_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [27]:
log_df.show(2, vertical=True, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------
 artist        | Harmonia                                                                                                                                  
 auth          | Logged In                                                                                                                                 
 firstName     | Ryan                                                                                                                                      
 gender        | M                                                                                                                                         
 itemInSession | 0                                                                                                                                         
 lastName      | Smith                                          

In [32]:
# Count value in page column
log_df.groupBy("page").count().orderBy(F.desc_nulls_first("count")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----+
|            page|count|
+----------------+-----+
|        NextSong| 6820|
|            Home|  806|
|           Login|   92|
|          Logout|   90|
|       Downgrade|   60|
|        Settings|   56|
|            Help|   47|
|           About|   36|
|         Upgrade|   21|
|   Save Settings|   10|
|           Error|    9|
|  Submit Upgrade|    8|
|Submit Downgrade|    1|
+----------------+-----+

In [33]:
log_df.where(F.col("page") == 'NextSong').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6820

In [34]:
# Just keep data of page = NextSong
log_df = log_df.where(F.col("page") == 'NextSong')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Write user table

In [35]:
users_cols = ['userId', 'firstName', 'lastName', 'gender', 'level']
users_table = log_df.select(users_cols).distinct()
#
# Write
users_table.write.mode("overwrite").parquet(path=s3_bucket + 'users')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Write time table

In [38]:
log_df.select((F.col('ts')/1000).cast("timestamp").alias('datetime')).show(1, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+
|datetime               |
+-----------------------+
|2018-11-15 00:30:26.796|
+-----------------------+
only showing top 1 row

In [42]:
log_df.withColumn('start_time', (log_df['ts']/1000).cast('timestamp')).select("start_time").show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+
|start_time             |
+-----------------------+
|2018-11-15 00:30:26.796|
|2018-11-15 00:41:21.796|
|2018-11-15 00:45:41.796|
|2018-11-15 03:44:09.796|
|2018-11-15 05:48:55.796|
+-----------------------+
only showing top 5 rows

In [45]:
log_df.select((F.col('ts')/1000).cast("timestamp").alias('start_time')).show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+
|start_time             |
+-----------------------+
|2018-11-15 00:30:26.796|
|2018-11-15 00:41:21.796|
|2018-11-15 00:45:41.796|
|2018-11-15 03:44:09.796|
|2018-11-15 05:48:55.796|
+-----------------------+
only showing top 5 rows

In [50]:
# Create new column with datetime type
log_df = log_df.withColumn("start_time", (F.col('ts')/1000).cast("timestamp").alias('start_time'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
# Extract information in start_time column
log_df = log_df.withColumn('weekday', F.date_format(F.col('start_time'), 'E'))
log_df = log_df.withColumn('year', F.year(F.col('start_time')))
log_df = log_df.withColumn('month', F.month(F.col('start_time')))
log_df = log_df.withColumn('week', F.weekofyear(F.col('start_time')))
log_df = log_df.withColumn('day', F.dayofmonth(F.col('start_time')))
log_df = log_df.withColumn('hour', F.hour(F.col('start_time')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
time_cols = ['start_time', 'weekday', 'year', 'month', 'week', 'day', 'hour']
time_table = log_df.select(time_cols).distinct()

# Write
time_table.write.mode('overwrite').partitionBy('year', 'month').parquet(path=s3_bucket + 'time')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Write Songplays table

In [72]:
song_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude', 'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year']

In [77]:
# Because song_df also has year column, we need to replace column year in song_df to another name before join two df
new_log_df = log_df.join(song_df.withColumnRenamed('year', 'year_song'), (log_df.song == song_df.title)\
                                   & (log_df.artist == song_df.artist_name)\
                                   & (log_df.length == song_df.duration), "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
# Check new dataframe of log is equal to origin
new_log_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6820

In [80]:
# Check list of column name in new dataframe
new_log_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'method', 'page', 'registration', 'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId', 'start_time', 'weekday', 'year', 'month', 'week', 'day', 'hour', 'artist_id', 'artist_latitude', 'artist_location', 'artist_longitude', 'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year_song']

In [78]:
songplays_cols = ['start_time', 'userId', 'level', 'song_id', 'artist_id', 'sessionId','location','userAgent', 'year', 'month']
songplays_table = new_log_df.select([F.col(x) for x in songplays_cols]).distinct()

# Write
songplays_table.write.mode("overwrite").partitionBy('year', 'month').parquet(path=s3_bucket + 'songplays')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…